<h1><center>GENE PREDICTION USING HIDDEN MARKOV MODEL</center></h1>

In [72]:
from Bio import SeqIO
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [53]:
def gene_predict_HMM_train(training_data):
    
    # Seperating intron and exon from the training data-set
    header_list = list(training_data['HEADER'].values)
    Sequence_list = list(training_data['SEQUENCE'].values)
    Exon = ''
    Intron = ''
    cnt_exon = 0 # Number of coding genes
    cnt_intron = 0 # Number of non-coding genes
    for i in range(len(header_list)):
        if 'exon' in header_list[i]:
            Exon = Exon + Sequence_list[i]
            cnt_exon = cnt_exon + 1
        elif 'intron' in header_list[i]:
            Intron = Intron + Sequence_list[i]
            cnt_intron = cnt_intron + 1
        else:
            None
       
    # Finding the length of the exon and intron
    exon_len = len(Exon)
    intron_len = len(Intron)

    # Nucleotides in the genome---------------------------------------------------------------------------
    Nucleotides = ['A','G','T','C']

    # EXON -----------------------------------------------------------------------------------------------
    # Frequencies  of the nucleotides present in Exon
    frq = []
    for n in Nucleotides:
        frq.append(Exon.count(n))

    # Probability of the nucleotides in Exon
    P_exon = []
    for r in frq:
        P_exon.append(r/exon_len)

    # INTRON ----------------------------------------------------------------------------------------------
    # Frequencies  of the nucleotides present in Intron
    frq1 = []
    for n1 in Nucleotides:
        frq1.append(Intron.count(n1))

    # Probability of the nucleotides in Intron
    P_intron = []
    for r1 in frq1:
        P_intron.append(r1/intron_len)
    
    # Probability of switching from intron to exon (probability of starting of a gene)
    P_intron_exon = cnt_exon/(exon_len+intron_len)
    
    # Probability of switching from exon to intron (probability of ending of a gene)
    P_exon_intron = P_intron_exon
    
    # Probability of swithcing from intron to intron
    P_intron_intron = 1 - P_intron_exon
    
    # Probability of swithcing from exon to exon
    P_exon_exon = 1 - P_intron_exon
    
    # Hidden states - transition matrix formation   
    genome = Exon + Intron
    
    # Finding the Transition probability
    P_A_A = genome.count('AA')/len(genome)
    P_A_T = genome.count('AT')/len(genome)
    P_A_G = genome.count('AG')/len(genome)
    P_A_C = genome.count('AC')/len(genome)

    P_G_A = genome.count('GA')/len(genome)
    P_G_T = genome.count('GT')/len(genome)
    P_G_G = genome.count('GG')/len(genome)
    P_G_C = genome.count('GC')/len(genome)

    P_C_A = genome.count('CA')/len(genome)
    P_C_T = genome.count('CT')/len(genome)
    P_C_G = genome.count('CG')/len(genome)
    P_C_C = genome.count('CC')/len(genome)

    P_T_A = genome.count('TA')/len(genome)
    P_T_T = genome.count('TT')/len(genome)
    P_T_G = genome.count('TG')/len(genome)
    P_T_C = genome.count('TC')/len(genome)
    
    # Graph creation
    G1 = nx.DiGraph()
    G1.add_nodes_from(('N','E'))
    weighted_edges1 = [('N','E',P_A_G),('E','N',P_A_C),('E','E',P_A_T),('N','N',P_A_A)]
    G1.add_weighted_edges_from(weighted_edges1)
    
    # Adjacency matrix formation
    Transition_Matrix = nx.linalg.graphmatrix.adjacency_matrix(G1).todense()
    
    return Transition_Matrix, P_intron_exon, P_exon_intron, P_intron_intron, P_exon_exon, P_exon, P_intron


In [54]:
# Using markov chain model computing 'pi' - By Eigen-values and Eigen-vectors
def compute_pi(Transition_matrix):   
    eig_val, eig_vec = np.linalg.eig(Transition_matrix)
    PI = np.real(eig_vec[:,0])
    return PI   

In [55]:
def gene_predict_HMM_test(testing_data):
    
    genome = testing_data
    Nucleotides = ['A','G','T','C']
    train_data = gene_predict_HMM_train(training_data)
    Transition_matrix, P_intron_exon, P_exon_intron, P_intron_intron, P_exon_exon, P_exon, P_intron = training[0], train_data[1], train_data[2], train_data[3], train_data[4], train_data[5], train_data[6]
    
    PI = compute_pi(Transition_matrix)
    alpha1 = PI[0]
    alpha2 = PI[1]
    n = 2 # Number of hidden states   
    string = ''
    for t in range(len(genome)): 
        curr_nuc = genome[t]
        idx = Nucleotides.index(curr_nuc)
        for I in range(n):
            alpha1 = alpha1*P_exon_exon*P_exon[idx] + alpha1*P_exon_intron*P_exon[idx]
            alpha2 = alpha2*P_intron_exon*P_intron[idx] + alpha1*P_intron_intron*P_intron[idx]
            #print(alpha1,alpha2)
        if alpha1 > alpha2:
            string = string + 'E'
        else:
            string = string + 'I' 
    predicted_seq = string
    return predicted_seq

In [104]:
# Training the algorithm for predicting the gene sequence
ESPN_data = pd.read_excel('Data-sets\\ESPN_RefGenome.xlsx')
ESPN_data = ESPN_data[0:27]
training_data = ESPN_data
#training_data

In [103]:
# Testing the data and finding the hidden states of the sequence
file_name = "C:\\Users\HP\\SEM 3 - PROJECTS\\19BIO201 - INTELLIGENCE OF BIOLOGICAL SYSTEMS\\Data-sets\\GRCH38_Testing_sequence.fa"
seq_obj = SeqIO.read(file_name,'fasta')
testing_data = str(seq_obj.seq)
tested_data = gene_predict_HMM_test(testing_data)
#tested_data

In [101]:
Predcited_gene_sequence = ''
for e in range(len(tested_data)):
    if tested_data[e] == 'E':
        Predcited_gene_sequence = Predcited_gene_sequence + testing_data[e]

In [102]:
print('Predicted_gene_sequence:\n\n',Predcited_gene_sequence)

Predicted_gene_sequence:

 AGCTACTTGGGAGGCTAAGGTGGGACGCTTGCTCGAGCACGGGAAGGGGAGGTTGCAGTGAGCCGATAACACACCACTGCACTTCCAGCCTAGGTGAGAGTGAGACCTTGTCTCAAAAAAACAAAAAGAAACATTAAATAATATCCTTAATATTGCAACTTAAGTGACAGCCCAGGATATATGAATTCCCTTGTAAGGTTTTCTTAACAAAACACCAGTCACATAAGTGCATTTTATTTTATAT
